In [1]:
#pip install googletrans

In [2]:
#pip install --upgrade certifi

In [3]:
#pip install fairseq

In [4]:
#pip install python-Levenshtein

In [5]:
#pip install sacremoses sentencepiece
#!pip install sacremoses

In [6]:
#pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

In [7]:
#!pip install pythainlp

In [8]:
#!pip install epitran

In [9]:
#pip install python-crfsuite

In [10]:
#pip install transformers

In [11]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib

# from geopy.distance import distance
import editdistance
import concurrent.futures

import re
from tqdm import tqdm

from googletrans import Translator
import pythainlp
from pythainlp import sent_tokenize,correct
from pythainlp.tokenize.multi_cut import find_all_segment, mmcut, segment
from pythainlp.soundex import lk82, metasound, udom83
from pythainlp.translate import Translate

from Levenshtein import distance
from Levenshtein import ratio

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Import data

In [12]:
smap_df = pd.read_excel('smap_baannia.xlsx', sheet_name='smap')
baannia_df = pd.read_excel('smap_baannia.xlsx', sheet_name='baannia')

In [13]:
smap_df.head(10)

,hs_nm
0,วงศ์ทองโคกเมา 2
1,ซิลวาโน คลองแห
2,ทรัพย์จุไร
3,ร่มฉัตร
4,ร่มรื่นริเวอร์ไซด์
5,ศิรดา แกรนด์ วิลล์
6,ศุภาลัย การ์เด้นวิลล์
7,Mplus
8,กุณสิริ
9,พฤกษา แกรนด์วิลล์


In [14]:
baannia_df.head(10)

,project_name_th
0,สีวลี สุวรรณภูมิ
1,บ้าน ณ ชเล เพลส
2,บ้านกัญญาเฮ้าส์ 3
3,บ้านเปี่ยมสุข 2
4,พฤกษาวิลล์ 38/1 หนามแดง-เทพารักษ์
5,บ้านพฤกษา 45 บางใหญ่
6,พนาสนธิ์ ซิตี้ ศรีนครินทร์
7,เพอร์เฟค พาร์ค บางบัวทอง
8,บ้านคุณาภัทร 5
9,บ้านเคหะธานี 4


# Processing data

In [15]:
smap_df.shape

(13581, 1)

In [16]:
baannia_df.shape

(8058, 1)

In [17]:
# check equivalence
print(lk82("กรูฟวิลล์ อยุธยา 3") == lk82("กรูฟวิล อยุธยา 3"))

True


In [18]:
re.sub(r"[^a-zA-Z0-9ก-๙]", "", "กรูฟวิลล์ อยุธยา 3")

'กรูฟวิลล์อยุธยา3'

In [19]:
smap_df["hs_name"] = smap_df.apply(lambda x: re.sub(r"[^a-zA-Z0-9ก-๙]", "", x["hs_nm"]),axis=1)
baannia_df["hs_name"] = baannia_df.apply(lambda x: re.sub(r"[^a-zA-Z0-9ก-๙]", "", x["project_name_th"]),axis=1)

In [20]:
smap_df.head(10)

,hs_nm,hs_name
0,วงศ์ทองโคกเมา 2,วงศ์ทองโคกเมา2
1,ซิลวาโน คลองแห,ซิลวาโนคลองแห
2,ทรัพย์จุไร,ทรัพย์จุไร
3,ร่มฉัตร,ร่มฉัตร
4,ร่มรื่นริเวอร์ไซด์,ร่มรื่นริเวอร์ไซด์
5,ศิรดา แกรนด์ วิลล์,ศิรดาแกรนด์วิลล์
6,ศุภาลัย การ์เด้นวิลล์,ศุภาลัยการ์เด้นวิลล์
7,Mplus,Mplus
8,กุณสิริ,กุณสิริ
9,พฤกษา แกรนด์วิลล์,พฤกษาแกรนด์วิลล์


In [21]:
baannia_df.tail(10)

,project_name_th,hs_name
8048,นินญา ราชพฤกษ์,นินญาราชพฤกษ์
8049,บ้านเวียงพิงค์วิลล่า,บ้านเวียงพิงค์วิลล่า
8050,สุรินทร์วิลเลจ,สุรินทร์วิลเลจ
8051,ดิ ไอดอล 1 เลียบคลองทวีวัฒนา,ดิไอดอล1เลียบคลองทวีวัฒนา
8052,เดอะ แพลนท์ ปากเกร็ด,เดอะแพลนท์ปากเกร็ด
8053,เดอะ เซ็นทรัล รีเจ้นท์ อัสสัมชัญ-ศรีราชา,เดอะเซ็นทรัลรีเจ้นท์อัสสัมชัญศรีราชา
8054,เดอะ เพลส กาญจนาภิเษก-ราชพฤกษ์,เดอะเพลสกาญจนาภิเษกราชพฤกษ์
8055,บ้านธนารักษ์ ชลบุรี,บ้านธนารักษ์ชลบุรี
8056,บ้านพัทธนันท์ สัตหีบ,บ้านพัทธนันท์สัตหีบ
8057,วิเศษสุขนคร โครงการ 27 - ประชาอุทิศ 90,วิเศษสุขนครโครงการ27ประชาอุทิศ90


In [22]:
distance("นินญาราชพฤกษ์","บ้านเวียงพิงค์วิลล่า")

17

In [23]:
baannia_df = baannia_df.drop_duplicates(subset=['hs_name'], inplace=False)

In [24]:
smap_df = smap_df.drop_duplicates(subset=['hs_name'], inplace=False)

In [25]:
dis_ls = []
shouse_ls = []
prev_house = ""
bhouse = "เดอะแพลนท์ปากเกร็ด"
for shouse in list(smap_df["hs_name"]):
    dis = distance(bhouse,shouse)
    dis_ls.append(dis)
    shouse_ls.append(shouse)
    


In [26]:
dis_ls.index(min(dis_ls))

3621

In [27]:
shouse_ls[3621]

'เดอะแพลนท์ปากเกร็ด'

In [28]:
smap_df.iloc[3621,0]

'เดอะ แพลนท์ ปากเกร็ด'

In [29]:
min_dis_ls = []
similar_hs_ls = []
similar_hs_org_ls = []
for bhouse in list(baannia_df["hs_name"]):
    dis_ls = []
    shouse_ls = []
    for shouse in list(smap_df["hs_name"]):
        dis = distance(bhouse,shouse)
        dis_ls.append(dis)
        shouse_ls.append(shouse)
    
    min_dis = min(dis_ls)
    simi_hs = shouse_ls[dis_ls.index(min(dis_ls))]
    simi_hs_org = smap_df.iloc[dis_ls.index(min(dis_ls)),0]
    
    min_dis_ls.append(min_dis)
    similar_hs_ls.append(simi_hs)
    similar_hs_org_ls.append(simi_hs_org)
    
    

In [30]:
len(min_dis_ls)

8040

In [31]:
len(similar_hs_ls)

8040

In [32]:
baannia_pred_df = baannia_df[:]

In [33]:
baannia_pred_df["min_dis"] = min_dis_ls
baannia_pred_df["pred_hs_nm_smap"] = similar_hs_ls
baannia_pred_df["pred_hs_nm_smap_org"] = similar_hs_org_ls

D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\741063588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baannia_pred_df["min_dis"] = min_dis_ls
D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\741063588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baannia_pred_df["pred_hs_nm_smap"] = similar_hs_ls
D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\741063588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [34]:
baannia_pred_df

,project_name_th,hs_name,min_dis,pred_hs_nm_smap,pred_hs_nm_smap_org
0,สีวลี สุวรรณภูมิ,สีวลีสุวรรณภูมิ,0,สีวลีสุวรรณภูมิ,สีวลี สุวรรณภูมิ
1,บ้าน ณ ชเล เพลส,บ้านณชเลเพลส,4,บ้านฉางเพลส,บ้านฉางเพลส
2,บ้านกัญญาเฮ้าส์ 3,บ้านกัญญาเฮ้าส์3,4,กัญญาเฮ้าส์3,กัญญาเฮ้าส์ 3
3,บ้านเปี่ยมสุข 2,บ้านเปี่ยมสุข2,1,บ้านเปี่ยมสุข,บ้านเปี่ยมสุข
4,พฤกษาวิลล์ 38/1 หนามแดง-เทพารักษ์,พฤกษาวิลล์381หนามแดงเทพารักษ์,8,พฤกษาวิลล์381หนามแดงบางนา,พฤกษาวิลล์ 38/1 หนามแดง-บางนา
...,...,...,...,...,...
8053,เดอะ เซ็นทรัล รีเจ้นท์ อัสสัมชัญ-ศรีราชา,เดอะเซ็นทรัลรีเจ้นท์อัสสัมชัญศรีราชา,7,เดอะเซ็นทรัลซิตี้อัสสัมชัญศรีราชา,เดอะ เซ็นทรัล ซิตี้ อัสสัมชัญ-ศรีราชา
8054,เดอะ เพลส กาญจนาภิเษก-ราชพฤกษ์,เดอะเพลสกาญจนาภิเษกราชพฤกษ์,5,เดอะวิลเลจกาญจนาภิเษกราชพฤกษ์,เดอะวิลเลจ กาญจนาภิเษก-ราชพฤกษ์
8055,บ้านธนารักษ์ ชลบุรี,บ้านธนารักษ์ชลบุรี,8,บ้านกรุงไทยชลบุรี,บ้านกรุงไทย ชลบุรี
8056,บ้านพัทธนันท์ สัตหีบ,บ้านพัทธนันท์สัตหีบ,10,บ้านพัธนชัย,บ้านพัธนชัย


In [35]:
min_dis_ls2 = []
similar_hs_ls2 = []
similar_hs_org_ls2 = []
for bhouse in list(baannia_df["hs_name"]):
    dis_ls = []
    shouse_ls = []
    for shouse in list(smap_df["hs_name"]):
        dis = ratio(bhouse,shouse)
        dis_ls.append(dis)
        shouse_ls.append(shouse)
    
    min_dis = max(dis_ls)
    simi_hs = shouse_ls[dis_ls.index(max(dis_ls))]
    simi_hs_org = smap_df.iloc[dis_ls.index(max(dis_ls)),0]
    
    min_dis_ls2.append(min_dis)
    similar_hs_ls2.append(simi_hs)
    similar_hs_org_ls2.append(simi_hs_org)

In [36]:
baannia_pred_df["max_ratio"] = min_dis_ls2
baannia_pred_df["pred_hs_nm_smap_rat"] = similar_hs_ls2
baannia_pred_df["pred_hs_nm_smap_org_rat"] = similar_hs_org_ls2

D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\1533037361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baannia_pred_df["max_ratio"] = min_dis_ls2
D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\1533037361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baannia_pred_df["pred_hs_nm_smap_rat"] = similar_hs_ls2
D:\Users\pimawat.m\AppData\Local\Temp\ipykernel_131048\1533037361.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [41]:
baannia_pred_df.head(20)

,project_name_th,hs_name,min_dis,pred_hs_nm_smap,pred_hs_nm_smap_org,max_ratio,pred_hs_nm_smap_rat,pred_hs_nm_smap_org_rat
0,สีวลี สุวรรณภูมิ,สีวลีสุวรรณภูมิ,0,สีวลีสุวรรณภูมิ,สีวลี สุวรรณภูมิ,1.000000,สีวลีสุวรรณภูมิ,สีวลี สุวรรณภูมิ
1,บ้าน ณ ชเล เพลส,บ้านณชเลเพลส,4,บ้านฉางเพลส,บ้านฉางเพลส,0.720000,บ้านณัชชาพลัส,บ้านณัชชา พลัส
2,บ้านกัญญาเฮ้าส์ 3,บ้านกัญญาเฮ้าส์3,4,กัญญาเฮ้าส์3,กัญญาเฮ้าส์ 3,0.857143,กัญญาเฮ้าส์3,กัญญาเฮ้าส์ 3
3,บ้านเปี่ยมสุข 2,บ้านเปี่ยมสุข2,1,บ้านเปี่ยมสุข,บ้านเปี่ยมสุข,0.962963,บ้านเปี่ยมสุข,บ้านเปี่ยมสุข
4,พฤกษาวิลล์ 38/1 หนามแดง-เทพารักษ์,พฤกษาวิลล์381หนามแดงเทพารักษ์,8,พฤกษาวิลล์381หนามแดงบางนา,พฤกษาวิลล์ 38/1 หนามแดง-บางนา,0.823529,พฤกษาวิลล์871เทพารักษ์,พฤกษาวิลล์ 87/1 เทพารักษ์
5,บ้านพฤกษา 45 บางใหญ่,บ้านพฤกษา45บางใหญ่,0,บ้านพฤกษา45บางใหญ่,บ้านพฤกษา 45 บางใหญ่,1.000000,บ้านพฤกษา45บางใหญ่,บ้านพฤกษา 45 บางใหญ่
6,พนาสนธิ์ ซิตี้ ศรีนครินทร์,พนาสนธิ์ซิตี้ศรีนครินทร์,9,อินนิซิโอศรีนครินทร์,อินนิซิโอ ศรีนครินทร์,0.727273,อินนิซิโอศรีนครินทร์,อินนิซิโอ ศรีนครินทร์
7,เพอร์เฟค พาร์ค บางบัวทอง,เพอร์เฟคพาร์คบางบัวทอง,0,เพอร์เฟคพาร์คบางบัวทอง,เพอร์เฟค พาร์ค บางบัวทอง,1.000000,เพอร์เฟคพาร์คบางบัวทอง,เพอร์เฟค พาร์ค บางบัวทอง
8,บ้านคุณาภัทร 5,บ้านคุณาภัทร5,1,บ้านคุณาภัทร7,บ้านคุณาภัทร 7,0.923077,บ้านคุณาภัทร7,บ้านคุณาภัทร 7
9,บ้านเคหะธานี 4,บ้านเคหะธานี4,4,เคหะธานี4,เคหะธานี 4,0.818182,เคหะธานี4,เคหะธานี 4


In [38]:
#baannia_pred_df.to_csv('baannia_smap_map_25042024.csv', index=False, encoding ='utf-8-sig')

# Translate

In [39]:
translator = Translator()

In [40]:
translator.translate('แอท ดรีม เฮฟเว่น')

ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)